In [1]:
!pip install tensorflow

In [2]:
import pandas as pd

from sentence_transformers import SentenceTransformer, models

2022-01-11 08:10:42.458111: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-11 08:10:42.458419: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
###### CREATE MODEL ######
max_seq_length = 128
train_batch_size = 64

# Load teacher model
print("Load teacher model")
teacher_model = SentenceTransformer('stsb-roberta-base-v2')

# Create student model
print("Create student model")
word_embedding_model = models.Transformer("xlm-roberta-base")

# Apply mean pooling to get one fixed sized sentence vector
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=True,
                               pooling_mode_cls_token=False,
                               pooling_mode_max_tokens=False)

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

Load teacher model
Create student model


Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
#### Read Datasets ####

df1 = pd.read_csv('EN_TL_compiled_10k.csv')
df2 = pd.read_csv('STS_TL_10k.csv')
#df3 = pd.read_csv('datasets/translate_s2.txt', delimiter = '\t', encoding = 'unicode_escape')


In [5]:
df1

tl  \
0     Ayon naman kay Bayan Muna Rep. Carlos Zarate, ...   
1     Ayon naman sa Philippine Embassy sa Washington...   
2     Paano maipagtatanggol ng mga ordinaryong mamam...   
3                                         SM by the bay   
4     MANILA, Philippines - Inirekomenda ng Departme...   
...                                                 ...   
9995  Nagpasaklolo na sa pulisya at Meralco ang Metr...   
9996                                       Okra P60 P50   
9997  "We thank the Supreme Court for putting an end...   
9998  Hinimok rin naman ng DILG chief ang mga nabikt...   
9999  Naideklara itong fire out dakong alas-10:40 ng...   

                                                     en  
0     According to Bayan Muna Rep. Carlos Zarate, Ch...  
1     According to the Philippine Embassy in Washing...  
2     How do ordinary citizens defend themselves fro...  
3                                        SM by the bay   
4     Manila, Philippines - The Department of Interi...  
...                                                 ...  
9995  The Metro Manila Development Authority (MMDA) ...  
9996                                       Okra p60 p50  
9997  "We Thank The Supreme Court For Putting An End...  
9998  The DILG Chief also urged the victims of Gavie...  
9999       It was declared fire out at around 10:40 am.  

[10000 rows x 2 columns]

In [6]:
df2

s1  \
0     Pinag-iingat ng Konsulado ng Pilipinas sa Duba...   
1     "Para bang iniuugoy kami sa duyan," kuwento ni...   
2     Agad namang nagkaloob ng ayuda ang pamahalaang...   
3     May malaking pananagutan umano ang gobyerno la...   
4     Inupakan ng isang militanteng kongresista si B...   
...                                                 ...   
8995  Samantala, panibagong 77 namang Filipino overs...   
8996  Kailangang amyendahan ang Local Government Cod...   
8997  Samantala, ang mga residenteng nawalan ng tira...   
8998  BASILAN AC/a,!aEURoe Sa kabila ng ipinapatupad...   
8999  Dinampot ng NBI-Environmental Crime Division a...   

                                                     s2  label  
0                -- JAI (@JaiPaller) September 13, 2019      1  
1     Idineklarang oldest person si Tajima pitong bu...      1  
2     Agad na inaresto si Magtira at dinala sa Crimi...      1  
3     Dahil nahihirapan na mapauwi ang mga kababayan...      0  
4     "The Philippine government conveys its appreci...      1  
...                                                 ...    ...  
8995  Walo naman ang namatay sa viral disease, na na...      0  
8996  Sinabi ni House Sergeant at Arms General Nicas...      1  
8997  Samantala, ipinag-utos ni Interior Secretary E...      1  
8998  Naalarma ang militar at pulisya sa Jolo, Sulu ...      0  
8999  Ayon sa NBI, walang mga kaukulang environmenta...      0  

[9000 rows x 3 columns]

In [7]:
#df3

In [8]:
from sentence_transformers.datasets import ParallelSentencesDataset
from torch.utils.data import DataLoader
from sentence_transformers import SentencesDataset, losses, evaluation, readers


###### Load train sets ######

train_reader = ParallelSentencesDataset(student_model=model, teacher_model=teacher_model)
train_reader.add_dataset(df1.values.tolist())
#load_data('translate_s1.txt', encoding= 'unicode_escape')
train_dataloader = DataLoader(train_reader, shuffle=True, batch_size=train_batch_size)
train_loss = losses.MSELoss(model=model)

In [9]:
###### Load test sets Mean Squared Error (MSE) measures ######

evaluators = []

test_mse = evaluation.MSEEvaluator(df1['en'].values.tolist(),df1['tl'].values.tolist(), teacher_model=teacher_model, batch_size=train_batch_size, show_progress_bar=True, write_csv = True)
evaluators.append(test_mse)

Batches:   0%|          | 0/157 [00:00<?, ?it/s]

In [10]:
###### Load dev sets for Semantic Textual Similarity (STS) data ######

evaluator_sts = evaluation.EmbeddingSimilarityEvaluator(df2['s1'], df2['s2'], df2['label'], batch_size=train_batch_size, show_progress_bar=True, write_csv=True)
evaluators.append(evaluator_sts)

In [11]:
import tensorflow.compat.v1 as tf

#gpus = tf.config.experimental.list_physical_devices('GPU')
#print(gpus)

#tf.config.set_visible_devices([], 'CPU') # hide the CPU
#tf.config.set_visible_devices(gpus[0], 'GPU') # unhide potentially hidden GPU
#tf.config.set_visible_devices(gpus[1], 'GPU') # unhide potentially hidden GPU
#tf.config.set_visible_devices(gpus[2], 'GPU') # unhide potentially hidden GPU
#tf.config.set_visible_devices(gpus[3], 'GPU') # unhide potentially hidden GPU

#tf.config.get_visible_devices()


tf.debugging.set_log_device_placement(True)
gpus = tf.config.list_logical_devices('GPU')
strategy = tf.distribute.MirroredStrategy(gpus)
tf.config.get_visible_devices()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


2022-01-11 08:33:45.793391: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-01-11 08:33:45.793418: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-01-11 08:33:45.793435: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (f6514415431e): /proc/driver/nvidia/version does not exist
2022-01-11 08:33:45.793651: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [ ]:
###### Train model ######

import datetime

#from numba import jit, cuda, numba

#@cuda.jit

#with tf.device("/gpu:3"):
with strategy.scope():
    output_path = "output/model-" + datetime.date.today().strftime("%Y-%m-%d")
    model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluation.SequentialEvaluator(evaluators, main_score_function=lambda scores: scores[-1]),
          epochs=100,
          evaluation_steps=1000,
          warmup_steps=10000,
          scheduler='warmupconstant',
          output_path=output_path,
          save_best_model=True,
          optimizer_params= {'lr': 2e-5, 'eps': 1e-6, 'correct_bias': False}
          )



Epoch:   0%|          | 0/100 [00:00<?, ?it/s]

Iteration:   0%|          | 0/86 [00:00<?, ?it/s]

/opt/conda/lib/python3.9/site-packages/sentence_transformers/SentenceTransformer.py:530: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  labels = torch.tensor(labels).to(self._target_device)


In [ ]:
evaluators

In [ ]:
import scipy.spatial

#Corpus with example sentences
corpusEN = df3['EN'][:20].values.tolist()
corpusTL = df3['TL'][:20].values.tolist()

with strategy.scope():
    corpusEN_embeddings = model.encode(corpusEN)
    corpusTL_embeddings = model.encode(corpusTL)

In [ ]:
queries = df1['EN'][88:89].values.tolist()
query_embeddings = model.encode(queries)

In [ ]:
closest_n = 5

with strategy.scope():
    for query, query_embedding in zip(queries, query_embeddings):
        distances = scipy.spatial.distance.cdist([query_embedding], corpusEN_embeddings, "cosine")[0]
    
        results = zip(range(len(distances)), distances)
        results = sorted(results, key=lambda x: x[1])
    
        print("\n=======\n")
        print("Query:", query)
        print("\nTop 5 most similar sentences in corpus:\n")

        for idx, distance in results[0:closest_n]:
            print(corpusEN[idx].strip(), "(Score: %.4f)" % (1-distance))

In [ ]:
queriesTL = df1['TL'][88:89].values.tolist()
#queriesTL = ['gobyerno.']
queryTL_embeddings = model.encode(queriesTL)

In [ ]:
closest_n = 5
for query, queryTL_embedding in zip(queriesTL, queryTL_embeddings):
    distances = scipy.spatial.distance.cdist([queryTL_embedding], corpusTL_embeddings, "cosine")[0]
    
    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])
    
    print("\n=======\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:\n")

    for idx, distance in results[0:closest_n]:
        print(corpusTL[idx].strip(), "(Score: %.4f)" % (1-distance))

### after 100 epocs of training: Query Search Result

In [ ]:
import scipy.spatial

#Corpus with example sentences
corpusEN = df3['EN'][:20].values.tolist()
corpusTL = df3['TL'][:20].values.tolist()

with strategy.scope():
    corpusEN_embeddings = model.encode(corpusEN)
    corpusTL_embeddings = model.encode(corpusTL)

In [ ]:
queries = df1['EN'][88:89].values.tolist()
query_embeddings = model.encode(queries)

In [ ]:
closest_n = 5

with strategy.scope():
    for query, query_embedding in zip(queries, query_embeddings):
        distances = scipy.spatial.distance.cdist([query_embedding], corpusEN_embeddings, "cosine")[0]
    
        results = zip(range(len(distances)), distances)
        results = sorted(results, key=lambda x: x[1])
    
        print("\n=======\n")
        print("Query:", query)
        print("\nTop 5 most similar sentences in corpus:\n")

        for idx, distance in results[0:closest_n]:
            print(corpusEN[idx].strip(), "(Score: %.4f)" % (1-distance))

In [ ]:
queriesTL = df1['TL'][88:89].values.tolist()
#queriesTL = ['Balita tungkol sa transportasyon, gobyerno ng Pilipinas at iba pa.']
queryTL_embeddings = model.encode(queriesTL)

In [ ]:
closest_n = 5
for query, queryTL_embedding in zip(queriesTL, queryTL_embeddings):
    distances = scipy.spatial.distance.cdist([queryTL_embedding], corpusTL_embeddings, "cosine")[0]
    
    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])
    
    print("\n=======\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:\n")

    for idx, distance in results[0:closest_n]:
        print(corpusTL[idx].strip(), "(Score: %.4f)" % (1-distance))